ver.0.3課題
scoreに応じた深さのある探索
類義語をひとまとめに(同じような言葉が複数ならばないようにする)
same Asによる同義語の探索→実用的ではない


ver0.2目標
depth:1で、キーワードと関連語の関連性をわかりやすく表で示す

In [130]:
!pip install SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper
sparql = SPARQLWrapper(endpoint='http://ja.dbpedia.org/sparql', returnFormat='json')

# def sort_name_ranking(results,name_count,name_type):
#     for i in range(len(results["results"]["bindings"])):
#         if len(name_count) > 0 and results["results"]["bindings"][i][name_type]["value"] in name_count:
#             name_count[results["results"]["bindings"][i][name_type]["value"]] += 1
#             continue
#         else:
#             name_count[results["results"]["bindings"][i][name_type]["value"]] = 1
#             continue
#     sorted_name_count = sorted(name_count.items(), key=lambda x: x[1], reverse=True)
#     # print(sorted_name_count)
#     return sorted_name_count

#作り直し
def summarize_result_by_count(results,query_type):
    counted_result = []
    for i in range(len(results["results"]["bindings"])):
        find_flag = False
        if len(counted_result) == 0:
            adding = results["results"]["bindings"][i]
            adding["count"] = 1
            # print("adding: " + str(i) + ":" + query_type)
            # print(adding)
            counted_result.append(adding)
            continue
        for j in range(len(counted_result)):
            #ここの分岐がおかしい？
            
            if counted_result[j][query_type]["value"] == results["results"]["bindings"][i][query_type]["value"]:
                counted_result[j]["count"] += 1
                print(results["results"]["bindings"][i].keys())
                print(results["results"]["bindings"][i])
                if "p" in results["results"]["bindings"][i].keys() and "value" in results["results"]["bindings"][i]["p"].keys():
                    counted_result[j]["path_list"].append(results["results"]["bindings"][i]["p"]["value"])
                
                # print(counted_result[j])
                find_flag = True
                break
            else:
                continue
        if find_flag == False:
            adding = results["results"]["bindings"][i]
            adding["count"] = 1
            if "p" in results["results"]["bindings"][i].keys() and "value" in results["results"]["bindings"][i]["p"].keys():
                adding["path_list"] = [results["results"]["bindings"][i]["p"]["value"]]
            else:
                adding["path_list"] = []
            # print("adding: " + str(i) + ":" + query_type)
            # print(adding)
            counted_result.append(adding)
        else:
            # print("value一致を発見 : " + query_type + " : " + str(counted_result[j][query_type]["value"]))
            continue
        
    # print("counted_result: " + query_type)
    # print(counted_result)
    output = sorted(counted_result,key=lambda x: x["count"],reverse=True)
    print("output: " + query_type)
    print(output)
    return output
            


#ディープラーニングの肝、重みづけ
def score(result_s,results_o):
    score = 5 * len(result_s["results"]["bindings"])+len(results_o["results"]["bindings"])
    return score


print("はじめまして！　私はPBLアシスタントです")
x = input("あなたが調べたい言葉はなんですか？")
print("あなたが調べたい言葉「" + x + "」について調べます")
query_sentence_s1 = "PREFIX dbpj: <http://ja.dbpedia.org/resource/> SELECT * where { dbpj:" + x + " ?p ?o . }"
query_sentence_s2 = "PREFIX dbo: <http://dbpedia.org/ontology/> SELECT * where { dbo:" + x + " ?p ?o . }"
query_sentence_s3 = "PREFIX dbp: <http://ja.dbpedia.org/property/> SELECT * where { dbp:" + x + " ?p ?o . }"
query_sentence_o1 = "PREFIX dbpj: <http://ja.dbpedia.org/resource/> SELECT * where { ?s ?p dbpj:" + x + " . }"
query_sentence_o2 = "PREFIX dbo: <http://dbpedia.org/ontology/> SELECT * where { ?s ?p dbo:" + x + " . }"
query_sentence_o3 = "PREFIX dbp: <http://ja.dbpedia.org/property/> SELECT * where { ?s ?p dbp:" + x + " . }"


print("searching for " + x + " in subject : " + x + "?p ?o")
# path_count_s = {}
# name_count_s = {}
sorted_path_ranking_s = []
sorted_name_ranking_s = []
sparql.setQuery(query_sentence_s1)
results_s1 = sparql.query().convert()
# print(sparql.query())
print(results_s1)
print("results_s1(http://ja.dbpedia.org/resource/" + x + "): " + str(len(results_s1["results"]["bindings"])))
sorted_path_ranking_s = summarize_result_by_count(results_s1,"p")
sorted_name_ranking_s = summarize_result_by_count(results_s1,"o")

sparql.setQuery(query_sentence_s2)
results_s2 = sparql.query().convert()
# print(sparql.query())
# print(results_s2)
print("results_s2(http://dbpedia.org/ontology/" + x + "): " +  str(len(results_s2["results"]["bindings"])))
sorted_path_ranking_s += summarize_result_by_count(results_s2,"p")
sorted_name_ranking_s += summarize_result_by_count(results_s2,"o")

sparql.setQuery(query_sentence_s3)
results_s3 = sparql.query().convert()
# print(sparql.query())
# print(results_s3)
print("results_s3(http://ja.dbpedia.org/property/" + x + "): " + str(len(results_s3["results"]["bindings"])))
sorted_path_ranking_s += summarize_result_by_count(results_s3,"p")
sorted_name_ranking_s += summarize_result_by_count(results_s3,"o")

print(sorted_path_ranking_s)
print(sorted_name_ranking_s)
print(" ")
if len(sorted_name_ranking_s) > 30:
    for i in range(30):
        print(str(sorted_name_ranking_s[i]["count"]) + "回:" + sorted_name_ranking_s[i]["o"]["value"].split("/")[-1] + " ( "+str(sorted_name_ranking_s[i]) + ")")


print(" ")
print("searching for " + x + " in object : ?s ?p " + x)
# path_count_o = {}
# name_count_o = {}
sorted_path_ranking_o = []
sorted_name_ranking_o = []
sparql.setQuery(query_sentence_o1)
results_o1 = sparql.query().convert()
# print(sparql.query())
print(results_o1)
print("results_o1(http://ja.dbpedia.org/resource/" + x + "): " + str(len(results_o1["results"]["bindings"])))
sorted_path_ranking_o = summarize_result_by_count(results_o1,"p")
sorted_name_ranking_o = summarize_result_by_count(results_o1,"s")

sparql.setQuery(query_sentence_o2)
results_o2 = sparql.query().convert()
# print(sparql.query())
# print(results_o2)
print("results_o2(http://dbpedia.org/ontology/" + x + "): " + str(len(results_o2["results"]["bindings"])))
sorted_path_ranking_o += summarize_result_by_count(results_o2,"p")
sorted_name_ranking_o += summarize_result_by_count(results_o2,"s")

sparql.setQuery(query_sentence_o3)
results_o3 = sparql.query().convert()
# print(sparql.query())
# print(results_o3)
print("results_o3(http://ja.dbpedia.org/property/" + x + "): " + str(len(results_o3["results"]["bindings"])))
sorted_path_ranking_o += summarize_result_by_count(results_o3,"p")
sorted_name_ranking_o += summarize_result_by_count(results_o3,"s")

print(sorted_path_ranking_o)
print(sorted_name_ranking_o)


############################################
# sorted listを横方向に結合する必要がある #

if len(sorted_name_ranking_o) > 30:
    for i in range(30):
        print(str(sorted_name_ranking_o[i]["count"]) + "回:" + sorted_name_ranking_o[i]["s"]["value"].split("/")[-1] + " ( "+str(sorted_name_ranking_o[i]) + ")")



print(" ")
print("## Same as 探索 ##")

find_flag = False
for result in [results_s1,results_s2,results_s3]:
    for i in range(len(result["results"]["bindings"])):
        if result["results"]["bindings"][i]["p"]["value"] == "http://www.w3.org/2002/07/owl#sameAs":
            print("→ " + x + " は " + result["results"]["bindings"][i]["o"]["value"].split("/")[-1] + " と同じです")
            find_flag = True
for results in [results_o1,results_o2,results_o3]:
    for i in range(len(results["results"]["bindings"])):
        if results["results"]["bindings"][i]["p"]["value"] == "http://www.w3.org/2002/07/owl#sameAs":
            print("→ " + x + " は " + results["results"]["bindings"][i]["s"]["value"].split("/")[-1] + " と同じです")
            find_flag = True
if find_flag == False:
    print("→ " + x + " は同じものが見つかりませんでした")
    


print(" ")
print("## 具体性をジャッジ ##")
if len(results_s1["results"]["bindings"])+len(results_s2["results"]["bindings"])+len(results_s3["results"]["bindings"]) > 0 and len(results_o1["results"]["bindings"])+len(results_o2["results"]["bindings"])+len(results_o3["results"]["bindings"])> 0 and 5 * len(results_s1["results"]["bindings"])+len(results_s2["results"]["bindings"])+len(results_s3["results"]["bindings"]) > len(results_o1["results"]["bindings"])+len(results_o2["results"]["bindings"])+len(results_o3["results"]["bindings"]): 
    print("→ " + x + " は具体的な言葉です")
else:
    print("→ " + x + " は抽象的な言葉です")
    print("  " + x + " は有名な言葉ですが、具体的ではないです ")
    print("  もう少し具体的な言葉について探してみましょう ")



print(" ")
print("## ALL PROCESS FINISHED ##")

Defaulting to user installation because normal site-packages is not writeable
はじめまして！　私はPBLアシスタントです
あなたが調べたい言葉「」について調べます
searching for  in subject : ?p ?o
{'head': {'link': [], 'vars': ['p', 'o']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
results_s1(http://ja.dbpedia.org/resource/): 0
output: p
[]
output: o
[]
results_s2(http://dbpedia.org/ontology/): 0
output: p
[]
output: o
[]
results_s3(http://ja.dbpedia.org/property/): 2
output: p
[{'p': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'o': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'}, 'count': 1}, {'p': {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#label'}, 'o': {'type': 'literal', 'xml:lang': 'ja', 'value': '1-'}, 'count': 1, 'path_list': ['http://www.w3.org/2000/01/rdf-schema#label']}]
output: o
[{'p': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'o': {'type': 'uri', 'value': 'http://www.w3.org/

colaboratoryで公開済み
ver0.1
https://colab.research.google.com/drive/13F11kCIz2aU-PQpQkFAXFNNDXtFfuggF?hl=ja#scrollTo=Y5WeR5pECE5A